In [7]:
import numpy as np
import pandas
import os
import json

In [32]:
def process_odb_json_for_runlog(fi):
    with open(fi,'r') as fin:
        odb = json.load(fin)
    # print(odb)
    runinfo = odb['Runinfo']
    output = {
        'Run':        runinfo['Run number'] ,
        'Type':       odb['Experiment']['Run Parameters']['Run Type'] ,
        'Comment':    odb['Experiment']['Run Parameters']['Comment'] ,
        'Shifters':   odb['Experiment']['Run Parameters']['Operator'] ,
        'Start Time': runinfo['Start time'] ,
        'Stop Time':  runinfo['Stop time'] ,
        'NSubruns':   int(odb['Logger']['Channels']['0']['Settings']['Current filename'].split("_")[-1].split(".mid")[0]),
    }
    return output

def create_updated_runlog(
    df=None, 
    midas_indir='/home/jlab/testbeam_example_files/online'
):
    '''reads in the .json files produced by midas and udpates the run log'''
    files = [os.path.join(midas_indir, x) for x in os.listdir(midas_indir) if '.json' in x[-7:] and 'last' not in x]
    if(df is None):
        df = pandas.DataFrame(columns=['Run', 'Type', 'Comment', 'Shifters', 'Start Time', 'Stop Time', 'NSubruns'])
    for i, fi in enumerate(files):
        # print(i, fi)
        run = int(fi.split("run")[1].split(".json")[0])
        if(run in df['Run']):
            # print("   -> Run found")
            continue
        else:
            # print("   -> Run not found in log, processing!")
            try:
                processed_odb = process_odb_json_for_runlog(fi)
            except:
                print("Warning: Unable to process:", fi)
                continue
            df.loc[-1] = processed_odb
            df.index = df.index + 1
            # print(processed_odb)
            # break
    df.sort_values(by='Run', inplace=True)
    return df


In [33]:
df = create_updated_runlog()
df

,Run,Type,Comment,Shifters,Start Time,Stop Time,NSubruns
22,281,C,"crystal 0 1050V, Co-60","Josh, Dave, Omar",Wed Oct 25 11:44:15 2023,Wed Oct 25 11:53:29 2023,48
24,282,C,"crystal 0 1050V, Co-60","Josh, Dave, Omar",Wed Oct 25 11:56:16 2023,Wed Oct 25 11:59:30 2023,17
19,283,C,"crystal 0 1200V, Co-60","Josh, Dave, Omar",Wed Oct 25 12:00:00 2023,Wed Oct 25 12:04:17 2023,23
44,284,C,"crystal 0 1300V, Co-60","Josh, Dave, Omar",Wed Oct 25 12:04:44 2023,Wed Oct 25 12:09:14 2023,24
9,285,C,"channel 0 1300V, Co-60","Josh, Dave, Omar",Wed Oct 25 12:46:43 2023,Wed Oct 25 12:46:47 2023,0
41,286,C,"channel 0 1300V, Co-60","Josh, Dave, Omar",Wed Oct 25 12:52:01 2023,Wed Oct 25 12:57:25 2023,28
54,287,C,"channel 1 1300V, Co-60","Josh, Dave, Omar",Wed Oct 25 12:58:41 2023,Wed Oct 25 13:04:23 2023,30
30,288,C,"channel 1 1300V, Co-60","Josh, Dave, Omar",Wed Oct 25 13:04:56 2023,Wed Oct 25 13:15:30 2023,57
17,289,C,"channel 2 1300V, Co-60","Josh, Dave, Omar",Wed Oct 25 13:18:47 2023,Wed Oct 25 13:25:39 2023,37
3,290,C,"channel 3 1255V, Co-60","Josh, Dave, Omar",Wed Oct 25 13:32:16 2023,Wed Oct 25 13:38:11 2023,31


In [34]:
df2 = create_updated_runlog(df)

In [8]:
with open("/home/jlab/testbeam_example_files/online/run00319.json", 'r') as fin: 
    odb = json.load(fin)
odb.keys()

dict_keys(['/MIDAS version', '/MIDAS git revision', '/filename', '/ODB path', 'Experiment', 'System', 'Programs', 'Logger', 'Runinfo', 'Alarms', 'Sequencer', 'Elog', 'History', 'Equipment', 'Test', 'WebServer'])

In [13]:
[x for x in odb['Runinfo'].keys() if '/key' not in x]

['State',
 'Online Mode',
 'Run number',
 'Transition in progress',
 'Start abort',
 'Requested transition',
 'Start time',
 'Start time binary',
 'Stop time',
 'Stop time binary']

5

In [20]:
print(json.dumps(odb['Experiment']['Run Parameters'],indent=3))

{
   "Comment/key": {
      "type": 12,
      "item_size": 128,
      "access_mode": 7,
      "last_written": 1698511427
   },
   "Comment": "cobolt 60 on ch0",
   "Operator/key": {
      "type": 12,
      "item_size": 32,
      "access_mode": 7,
      "last_written": 1698511427
   },
   "Operator": "Josh, Omar",
   "Run Type/key": {
      "type": 3,
      "access_mode": 7,
      "last_written": 1698511427
   },
   "Run Type": "C"
}


In [16]:
print(json.dumps(odb['Runinfo'],indent=3))

{
   "State/key": {
      "type": 7,
      "access_mode": 7,
      "last_written": 1698511431
   },
   "State": 3,
   "Online Mode/key": {
      "type": 7,
      "access_mode": 7,
      "last_written": 1697739971
   },
   "Online Mode": 1,
   "Run number/key": {
      "type": 7,
      "access_mode": 7,
      "last_written": 1698511427
   },
   "Run number": 319,
   "Transition in progress/key": {
      "type": 7,
      "access_mode": 7,
      "last_written": 1698511494
   },
   "Transition in progress": 2,
   "Start abort/key": {
      "type": 7,
      "access_mode": 7,
      "last_written": 1698511494
   },
   "Start abort": 0,
   "Requested transition/key": {
      "type": 7,
      "access_mode": 7,
      "last_written": 1698443503
   },
   "Requested transition": 0,
   "Start time/key": {
      "type": 12,
      "item_size": 32,
      "access_mode": 7,
      "last_written": 1698511427
   },
   "Start time": "Sat Oct 28 09:43:47 2023",
   "Start time binary/key": {
      "type": 6,
 

In [12]:
odb['Runinfo'][]

KeyError: 'Comment'

In [ ]:

def create_updated_subrun_list(
    file_indir='/home/jlab/testbeam_example_files/nearline'
    log_indir='/home/jlab/testbeam_example_files/nearline_logs'
):
    '''looks for files in the specified directories and creates a list of files for jsroot to open'''